In [3]:
import os
import pandas as pd
import numpy as np
import cv2 as cv

DIR_DATASET = 'dataset'
DIR_EDIBLE_MUSHROOM = os.path.join(DIR_DATASET, 'EdibleMushrooms', 'data')
DIR_POISONOUS_MUSHROOM = os.path.join(DIR_DATASET, 'PoisonousMushrooms', 'data')

# **Load data**

In [2]:
list_folder =[["Agaricus bisporus",
                "Pleurotus ostreatus",
                "Pleurotus eryngii",
                "Cantharellus cibarius",
                "Boletus edulis",
                "Hydnum repandum",
                "Laetiporus",
                "Craterellus cornucopioides",
                "Clitocybe nuda",
                "Morchella",
                "Flammulina velutipes",
                "Hypsizygus marmoreus",
                "Lentinula edodes",
                "Grifola frondosa",
                "Ganoderma lingzhi",
                "Hericium erinaceus",
                "Tricholoma matsutake",
                "Calvatia gigantea"],
                ["Amanita phalloides",
                "Amanita virosa",
                "Gyromitra esculenta",
                "Galerina Marginata",
                "Clitocybe dealbata",
                "Cortinarius rubellus",
                "Amanita verna",
                "Podostroma Cornu-damae",
                "Pleurocybella porrigens",
                "Amanita muscaria"]]

X_data = [] 
y_data = []

# Load tất cả hình ảnh trong thư mục
total = 0
print("[INFO] - Loading data...")
for idx, scientific_names in enumerate(list_folder):
    for item in scientific_names:
        path_folder =  os.path.join(DIR_EDIBLE_MUSHROOM if idx == 0 else DIR_POISONOUS_MUSHROOM, item)
        files = [f for f in os.listdir(path_folder) if os.path.isfile(os.path.join(path_folder, f))]
        i = 0
        for file in files:
            img = cv.imread(os.path.join(path_folder, file))
            if img is None:
                continue
            # img = cv.resize(img,(227, 227))
            X_data.append(img)
            y_data.append('EDIBLE' if idx == 0 else 'POISONOUS_' + item)
            i += 1
            total += 1
        print(f"\t{item}: {i} loaded")

print("[INFO] - Total image:", total)

[INFO] - Loading data...
	Agaricus bisporus: 75 loaded
	Pleurotus ostreatus: 190 loaded
	Pleurotus eryngii: 51 loaded
	Cantharellus cibarius: 241 loaded
	Boletus edulis: 191 loaded
	Hydnum repandum: 266 loaded
	Laetiporus: 348 loaded
	Craterellus cornucopioides: 343 loaded
	Clitocybe nuda: 273 loaded
	Morchella: 251 loaded
	Flammulina velutipes: 268 loaded
	Hypsizygus marmoreus: 67 loaded
	Lentinula edodes: 70 loaded
	Grifola frondosa: 246 loaded
	Ganoderma lingzhi: 89 loaded
	Hericium erinaceus: 201 loaded
	Tricholoma matsutake: 157 loaded
	Calvatia gigantea: 150 loaded
	Amanita phalloides: 281 loaded
	Amanita virosa: 285 loaded
	Gyromitra esculenta: 330 loaded
	Galerina Marginata: 357 loaded
	Clitocybe dealbata: 338 loaded
	Cortinarius rubellus: 351 loaded
	Amanita verna: 188 loaded
	Podostroma Cornu-damae: 53 loaded
	Pleurocybella porrigens: 385 loaded
	Amanita muscaria: 341 loaded
[INFO] - Total image: 6386


In [28]:
# Lưu hình ảnh đã load vào các thư mục tương ứng mỗi nhãn
i = 0
df = {'Name':[], 'Label':[]}
label_current = y_data[0].replace(' ', '_').upper()
if not os.path.exists(f'data/{label_current}'):
    os.mkdir(f'data/{label_current}')
for img, label in zip(X_data, y_data):
    label = label.replace(' ', '_').upper()
    if label != label_current:
        i = 0
        if not os.path.exists(f'data/{label}'):
            os.mkdir(f'data/{label}')
    cv.imwrite(f"data/{label}/{i}.jpg", img)
    df['Name'].append(f"{i}.jpg")
    df["Label"].append(label)
    i += 1
    label_current = label

In [36]:
df['Label'].replace('POISONOUS_PODOSTROMA_CORNU-DAMAE', 'POISONOUS_PODOSTROMA_CORNUDAMAE', inplace=True)

In [32]:
i = 0
df = {'Name':[], 'Label':[]}
label_current = y_data[0].replace(' ', '_').upper()
for img, label in zip(X_data, y_data):
    label = label.replace(' ', '_').upper()
    if label != label_current:
        i = 0
    df['Name'].append(f"{i}.jpg")
    df["Label"].append(label)
    i += 1
    label_current = label

In [ ]:
df = pd.DataFrame(df)
df.to_csv('data/mushroom_dataset.csv', index=False)

In [43]:
df.head()

,Name,Label
0,0.jpg,EDIBLE
1,1.jpg,EDIBLE
2,2.jpg,EDIBLE
3,3.jpg,EDIBLE
4,4.jpg,EDIBLE


In [41]:
df['Label'].value_counts()

EDIBLE                               3477
POISONOUS_PLEUROCYBELLA_PORRIGENS     385
POISONOUS_GALERINA_MARGINATA          357
POISONOUS_CORTINARIUS_RUBELLUS        351
POISONOUS_AMANITA_MUSCARIA            341
POISONOUS_CLITOCYBE_DEALBATA          338
POISONOUS_GYROMITRA_ESCULENTA         330
POISONOUS_AMANITA_VIROSA              285
POISONOUS_AMANITA_PHALLOIDES          281
POISONOUS_AMANITA_VERNA               188
POISONOUS_PODOSTROMA_CORNUDAMAE        53
Name: Label, dtype: int64

# **Split data**

In [17]:
df = pd.read_csv('data/mushroom_dataset.csv')
df.sample(5)

,Name,Label
5928,268.jpg,POISONOUS_PLEUROCYBELLA_PORRIGENS
5984,324.jpg,POISONOUS_PLEUROCYBELLA_PORRIGENS
313,313.jpg,EDIBLE
5532,113.jpg,POISONOUS_AMANITA_VERNA
700,700.jpg,EDIBLE


## **Chia dữ liệu thành 2 loại nấm ăn được và nấm độc**

In [22]:
# Tạo thư mục gốc để chia tập dữ liệu
if not os.path.exists('data_splited_general'):
    os.mkdir('data_splited_general')
# Tạo các thư mục train, dev, test nếu không tồn tại
for folder_set in ['train', 'dev', 'test']:
    if not os.path.exists(f'data_splited_general/{folder_set}'):
        os.mkdir(f'data_splited_general/{folder_set}')
    # Tạo các thư mục con trong các tập dữ liệu
    for sub_folder in ['EDIBLE', 'POISONOUS']:
        if not os.path.exists(f'data_splited_general/{folder_set}/{sub_folder}'):
            os.mkdir(f'data_splited_general/{folder_set}/{sub_folder}')

In [25]:
from sklearn.model_selection import train_test_split

# Chia tập dữ liệu thành train, dev, test và lưu vào thư mục
for item in df['Label'].unique():
    print(f'Spliting {item}...')
    df_item = df[df['Label'] == item]

    df_train_item, df_test_item = train_test_split(
        df_item, test_size=0.2, random_state=42)
    df_train_item, df_dev_item = train_test_split(
        df_train_item, test_size=0.1, random_state=42)

    for set_name, df_set in zip(['train', 'dev', 'test'], [df_train_item, df_dev_item, df_test_item]):
        for index, row in df_set.iterrows():
            path_file = os.path.join('data', row['Label'], row['Name'])
            img = cv.imread(path_file)
            if img is None:
                print('ERROR - Image is none!')
                continue
            cv.imwrite(os.path.join('data_splited_general', set_name,
                       'EDIBLE' if row['Label'] == 'EDIBLE' else 'POISONOUS', row['Name'] + '_' + row['Label']), img)


Spliting EDIBLE...
Spliting POISONOUS_AMANITA_PHALLOIDES...
Spliting POISONOUS_AMANITA_VIROSA...
Spliting POISONOUS_GYROMITRA_ESCULENTA...
Spliting POISONOUS_GALERINA_MARGINATA...
Spliting POISONOUS_CLITOCYBE_DEALBATA...
Spliting POISONOUS_CORTINARIUS_RUBELLUS...
Spliting POISONOUS_AMANITA_VERNA...
Spliting POISONOUS_PODOSTROMA_CORNUDAMAE...
Spliting POISONOUS_PLEUROCYBELLA_PORRIGENS...
Spliting POISONOUS_AMANITA_MUSCARIA...


In [ ]:
['EDIBLE', 'POISONOUS_AMANITA_PHALLOIDES',
 'POISONOUS_AMANITA_VIROSA', 'POISONOUS_GYROMITRA_ESCULENTA',
 'POISONOUS_GALERINA_MARGINATA', 'POISONOUS_CLITOCYBE_DEALBATA',
 'POISONOUS_CORTINARIUS_RUBELLUS', 'POISONOUS_AMANITA_VERNA',
 'POISONOUS_PODOSTROMA_CORNUDAMAE', 'POISONOUS_PLEUROCYBELLA_PORRIGENS',
 'POISONOUS_AMANITA_MUSCARIA']


In [26]:
df.Label.unique()

array(['EDIBLE', 'POISONOUS_AMANITA_PHALLOIDES',
       'POISONOUS_AMANITA_VIROSA', 'POISONOUS_GYROMITRA_ESCULENTA',
       'POISONOUS_GALERINA_MARGINATA', 'POISONOUS_CLITOCYBE_DEALBATA',
       'POISONOUS_CORTINARIUS_RUBELLUS', 'POISONOUS_AMANITA_VERNA',
       'POISONOUS_PODOSTROMA_CORNUDAMAE',
       'POISONOUS_PLEUROCYBELLA_PORRIGENS', 'POISONOUS_AMANITA_MUSCARIA'],
      dtype=object)

## **Chia dữ liệu nấm ăn được và các loài nấm độc riêng biệt** 

In [15]:
# Tạo thư mục gốc để chia tập dữ liệu
if not os.path.exists('data_splited_species'):
    os.mkdir('data_splited_species')
# Tạo các thư mục train, dev, test nếu không tồn tại
for folder_set in ['train', 'dev', 'test']:
    if not os.path.exists(f'data_splited_species/{folder_set}'):
        os.mkdir(f'data_splited_species/{folder_set}')
    # Tạo các thư mục con trong các tập dữ liệu
    for sub_folder in df['Label'].unique():
        if not os.path.exists(f'data_splited_species/{folder_set}/{sub_folder}'):
            os.mkdir(f'data_splited_species/{folder_set}/{sub_folder}')


In [16]:
from sklearn.model_selection import train_test_split

# Chia tập dữ liệu thành train, dev, test và lưu vào thư mục
for item in df['Label'].unique():
    print(f'Spliting {item}...')
    df_item = df[df['Label'] == item]

    df_train_item, df_test_item = train_test_split(
        df_item, test_size=0.2, random_state=42)
    df_train_item, df_dev_item = train_test_split(
        df_train_item, test_size=0.1, random_state=42)

    for set_name, df_set in zip(['train', 'dev', 'test'], [df_train_item, df_dev_item, df_test_item]):
        for index, row in df_set.iterrows():
            path_file = os.path.join('data', row['Label'], row['Name'])
            img = cv.imread(path_file)
            if img is None:
                print('ERROR - Image is none!')
                continue
            cv.imwrite(os.path.join('data_splited_species',
                       set_name, row['Label'], row['Name']), img)


Spliting EDIBLE...
Spliting POISONOUS_AMANITA_PHALLOIDES...
Spliting POISONOUS_AMANITA_VIROSA...
Spliting POISONOUS_GYROMITRA_ESCULENTA...
Spliting POISONOUS_GALERINA_MARGINATA...
Spliting POISONOUS_CLITOCYBE_DEALBATA...
Spliting POISONOUS_CORTINARIUS_RUBELLUS...
Spliting POISONOUS_AMANITA_VERNA...
Spliting POISONOUS_PODOSTROMA_CORNUDAMAE...
Spliting POISONOUS_PLEUROCYBELLA_PORRIGENS...
Spliting POISONOUS_AMANITA_MUSCARIA...
